# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
!pip install langchain_community langchain_experimental langchain_openai

In [2]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [3]:
!pip install faiss-cpu > /dev/null
!pip install google-search-results > /dev/null

In [4]:

from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [6]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [12]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

game_prompt = PromptTemplate.from_template(
    "You are a video game reviewer skilled at analyzing and evaluating games. Talk about the name of the game, the number of users, and its story. Create one short report for this objective: {objective}"
)
game_chain = LLMChain(llm=OpenAI(temperature=0), prompt=game_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="VideoGameReviewer",
        func=game_chain.run,
        description="Useful when you need to create short reports. Input: A target to create a report for. Output: A report for this target. Please specify the target precisely!"
    ),
]


prefix = """You are an AI who performs one report based on the following objective: {objective}. Consider previous reports to avoid duplication : {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [13]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [14]:
OBJECTIVE = "Write a short report on the most popular video game of 2024"

In [15]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
# max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=1,
)

In [16]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: To create a report on the most popular video game of 2024, I need to gather information and analyze data.
Action: Search
Action Input: "most popular video game 2024"
Observation: {'type': 'organic_result'}
Thought: I should also consider previous reports to avoid duplication.
Action: VideoGameReviewer
Action Input: "most popular video game 2024"
Observation: 


Report: Most Popular Video Game of 2024

As a video game reviewer, I have had the opportunity to play and analyze numerous games over the years. However, one game that has stood out to me as the most popular video game of 2024 is "Galactic Odyssey."

The name itself evokes a sense of adventure and exploration, which perfectly captures the essence of the game. Developed by a team of talented developers, "Galactic Odyssey" has taken the gaming world by storm, with millions of users worldwide.

The ga

{'objective': 'Write a short report on the most popular video game of 2024'}